# Part 2: Scrape and Analyze Mars Weather Data

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser

In [3]:
# URL to scrape
url = 'https://static.bc-edx.com/data/web/mars_facts/temperature.html'

# Define the browser to utilize
browser = Browser('chrome')

# Visit the browser
browser.visit(url)

# Return all the HTML on our page
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, "html.parser")

In [5]:
# Find the table
table = soup.find('table', class_='table')

In [22]:
# Store the table
mars = []
rows = table.find_all('tr')


# Loop through the rows

df = pd.DataFrame(rows)
print(df)

      0       1   2                   3   4       5   6      7   8        9   \
0     \n    [id]  \n  [terrestrial_date]  \n   [sol]  \n   [ls]  \n  [month]   
1     \n     [2]  \n        [2012-08-16]  \n    [10]  \n  [155]  \n      [6]   
2     \n    [13]  \n        [2012-08-17]  \n    [11]  \n  [156]  \n      [6]   
3     \n    [24]  \n        [2012-08-18]  \n    [12]  \n  [156]  \n      [6]   
4     \n    [35]  \n        [2012-08-19]  \n    [13]  \n  [157]  \n      [6]   
...   ..     ...  ..                 ...  ..     ...  ..    ...  ..      ...   
1863  \n  [1889]  \n        [2018-02-23]  \n  [1973]  \n  [133]  \n      [5]   
1864  \n  [1892]  \n        [2018-02-24]  \n  [1974]  \n  [134]  \n      [5]   
1865  \n  [1894]  \n        [2018-02-25]  \n  [1975]  \n  [134]  \n      [5]   
1866  \n  [1893]  \n        [2018-02-26]  \n  [1976]  \n  [135]  \n      [5]   
1867  \n  [1895]  \n        [2018-02-27]  \n  [1977]  \n  [135]  \n      [5]   

      10          11  12          13  1

C:\Users\mochi\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
